In [6]:
import os
import cv2
import numpy as np

In [1]:
def flip(image, flip_code):
    return cv2.flip(image, flip_code)

def rotate(image, angle):
    rows, cols = image.shape[:2]
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
    return cv2.warpAffine(image, M, (cols, rows))

def adjust_brightness(image, delta):
    return np.clip(image + delta, 0, 255).astype(np.uint8)

def motion_blur(image, kernel_size=25):
    kernel_motion_blur = np.zeros((kernel_size, kernel_size))
    kernel_motion_blur[int((kernel_size-1)/2), :] = np.ones(kernel_size)
    kernel_motion_blur = kernel_motion_blur / kernel_size
    return cv2.filter2D(image, -1, kernel_motion_blur)

def shear(image, shear_factor):
    rows, cols = image.shape[:2]
    shear_matrix = np.array([[1, shear_factor, 0], [0, 1, 0]])
    return cv2.warpAffine(image, shear_matrix, (cols, rows), borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))

def grid_distortion(image, grid_size=4, magnitude=1):
    rows, cols = image.shape[:2]
    x_points = np.linspace(0, cols, grid_size+1, dtype=np.float32)
    y_points = np.linspace(0, rows, grid_size+1, dtype=np.float32)
    x_mesh, y_mesh = np.meshgrid(x_points, y_points)
    x_mesh_flat = x_mesh.flatten()
    y_mesh_flat = y_mesh.flatten()

    x_mesh_flat += np.random.normal(0, magnitude, size=x_mesh_flat.shape)
    y_mesh_flat += np.random.normal(0, magnitude, size=y_mesh_flat.shape)
    
    map_x = np.clip(cv2.resize(x_mesh_flat.reshape(grid_size+1, grid_size+1), (cols, rows)).astype(np.float32), 0, cols-1)
    map_y = np.clip(cv2.resize(y_mesh_flat.reshape(grid_size+1, grid_size+1), (cols, rows)).astype(np.float32), 0, rows-1)
    
    distorted_image = cv2.remap(image, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
    
    return distorted_image

def adjust_contrast(image, contrast_type='increase', alpha=1.5, beta=0):
    if contrast_type == 'increase':
        adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    elif contrast_type == 'decrease':
        adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    else:
        raise ValueError("Invalid contrast_type. Use 'increase' or 'decrease'.")
    return adjusted_image


In [2]:
def apply_augmentations_to_images(root_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                file_path = os.path.join(subdir, file)
                image = cv2.imread(file_path)
                if image is None:
                    continue

                augmented_images = [
                    ('flip_horizontally', flip(image, 1)),
                    ('flip_vertically', flip(image, 0)),
                    ('rotate_90', rotate(image, 90)),
                    ('rotate_180', rotate(image, 180)),
                    # ('brightness_increase', adjust_brightness(image, 50)),
                    ('brightness_decrease', adjust_brightness(image, -50)),
                    ('motion_blur', motion_blur(image)),
                    ('shear', shear(image, 0.2)),
                    ('grid_distortion', grid_distortion(image)),
                    ('contrast_increase', adjust_contrast(image, 'increase')),
                    ('contrast_decrease', adjust_contrast(image, 'decrease', alpha=0.5))
                ]

                for aug_name, aug_image in augmented_images:
                    relative_path = os.path.relpath(subdir, root_dir)
                    output_subdir = os.path.join(output_dir, relative_path)
                    if not os.path.exists(output_subdir):
                        os.makedirs(output_subdir)
                    
                    base_name, ext = os.path.splitext(file)
                    output_file_path = os.path.join(output_subdir, f"{base_name}_{aug_name}{ext}")
                    cv2.imwrite(output_file_path, aug_image)

In [7]:
path_to_root = r'C:\Users\Groot\Code\oral_cancer\modified\complete_dsm'
path_to_output =r'C:\Users\Groot\Code\oral_cancer\modified\complete_dsm_aug'

In [4]:
apply_augmentations_to_images(path_to_root, path_to_output)

In [5]:
import albumentations as A

augmentations = [
    ("flip_horizontal", A.HorizontalFlip(p=1)),
    ("flip_vertical", A.VerticalFlip(p=1)),
    ("shift_scale_rotate", A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=45, p=1)),
    ("motion_blur", A.MotionBlur(blur_limit=(3, 7), p=1)),
    ("brightness_contrast", A.RandomBrightnessContrast(p=1)),
    ("grid_distortion", A.GridDistortion(p=1)),
    ("hue_saturation", A.HueSaturationValue(p=1)),
    ("elastic_transform", A.ElasticTransform(p=1))
]


def apply_augmentations_to_images2(root_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                file_path = os.path.join(subdir, file)
                image = cv2.imread(file_path)
                if image is None:
                    continue

                for aug_name, augmentation in augmentations:
                    augmented = augmentation(image=image)
                    aug_image = augmented["image"]

                    relative_path = os.path.relpath(subdir, root_dir)
                    output_subdir = os.path.join(output_dir, relative_path)
                    if not os.path.exists(output_subdir):
                        os.makedirs(output_subdir)

                    base_name, ext = os.path.splitext(file)
                    output_file_path = os.path.join(output_subdir, f"{base_name}_{aug_name}{ext}")
                    cv2.imwrite(output_file_path, aug_image)





In [8]:
path_to_output2 =r'C:\Users\Groot\Code\oral_cancer\modified\complete_dsm_aug2'
apply_augmentations_to_images2(path_to_root, path_to_output2)